# Pre-processing (state / organization Name) Allocation data for WaDE upload.
- Purpose:  To pre-process the data into one master file for simple DataFrame creation and extraction

In [1]:
# Needed Libraries / Modules

# ---- working with data ----
import os  # native operating system interaction
import numpy as np  # mathematical array manipulation
import pandas as pd  # data structure and data analysis
import geopandas as gpd  # geo-data structure and data analysis

# ---- visualization ----
import matplotlib.pyplot as plt  # plotting library
import seaborn as sns  # plotting library

# ---- API data retrieval ----
import requests  # http requests
import json  # JSON parse

# ---- Cleanup ----
import re  # string regular expression manipulation
from datetime import datetime  # date and time manipulation
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x)  # suppress scientific notation in Pandas

In [2]:
# Working Directory

# set working directory, if need be
workingDir = "RawInputData" # file location
os.chdir(workingDir)

print(os.getcwd())

C:\Users\rjame\Documents\WSWC Documents\MappingStatesDataToWaDE2.0\Alaska\WaterAllocation\RawInputData


## Point of Diversion Data

In [3]:
# Input File  For Alaskas Points 
PodSubsurfacePoint = "shapefiles/SubsurfaceWaterRight_Point.csv"
PodSubsurfacePointFile = pd.read_csv(PodSubsurfacePoint, encoding = "ISO-8859-1").replace(np.nan, "")

PodSurfacePoint = "shapefiles/SurfaceWaterRight_Point.csv"
PodSurfacePointFile = pd.read_csv(PodSurfacePoint, encoding = "ISO-8859-1").replace(np.nan, "")

PodSurfacePointFile= PodSurfacePointFile.groupby('CASE_ID').agg(lambda x: ','.join([str(elem) for elem in (list(set(x))) if elem != ""])).replace(np.nan, "").reset_index()   
pod = pd.concat([PodSubsurfacePointFile,PodSurfacePointFile], axis = 0)
pod = pod.reset_index(drop=True)

#FI_PoD = "RawInputData/shapefiles/SubsurfaceWaterRight_Point.zip"
#dfinPOD = pd.read_csv(FI_PoD, encoding = "ISO-8859-1").replace(np.nan, "")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in pod:
    pod['WaDEUUID'] = "utD" + pod.index.astype(str)
    pod.to_csv('CombinedPOD_Points.zip', compression=dict(method='zip', archive_name='CombinedPOD_Points.csv'), index=False)
pd.set_option('display.max_columns', None)
print(len(pod))
pod.head()

18879


,CASE_ID,CSTMRNM,CSTYPDSCRP,SPCLCDDSCR,CSSTTSDSCR,FILE_TYPE,FILENUMBER,CASE_TYPE,SPCLCD,CASESTATUS,CSTMRLSTNM,CSTMRFRSTN,CSTMRMDDLN,CTSCLABEL,CS_LABEL,MI_LABEL,DISPLAY,INFO_LINK,RFRSHDT,GLOBALID,MV_OID,LASTEDIT,OPR,latitude,longitude,WaDEUUID
0,LAS 32578,Bitter Daisy Lee,Water Rights (801),Subsurface (SUB),Certificate Issued (36),LAS,32578,801,SUB,36,BITTER,DAISY,LEE,SUBSURFACE,CERT ISSUED,LAS 32578\nCERT ISSUED,Y,http://dnr.alaska.gov/projects/las/#filenumber...,"""2021-11-30T00:00:00.000Z""",{4F7A9F67-AFE1-4A1E-9782-C5C907BC83BE},1111042,20180906174756,WANC,59.68510,-151.47025,utD0
1,LAS 633,Black Dennis J,Water Rights (801),Subsurface (SUB),Certificate Issued (36),LAS,633,801,SUB,36,BLACK,DENNIS,J,SUBSURFACE,CERT ISSUED,LAS 633\nCERT ISSUED,Y,http://dnr.alaska.gov/projects/las/#filenumber...,"""2021-12-03T00:00:00.000Z""",{9D841C6A-9FF1-44F4-8774-EB597AB22671},1100529,20211202003245,WANC,61.07268,-149.77132,utD1
2,LAS 7457,Mierzejewski David W,Water Rights (801),Subsurface (SUB),Certificate Issued (36),LAS,7457,801,SUB,36,MIERZEJEWSKI,DAVID,W,SUBSURFACE,CERT ISSUED,LAS 7457\nCERT ISSUED,Y,http://dnr.alaska.gov/projects/las/#filenumber...,"""2021-12-03T00:00:00.000Z""",{BA9E8E57-0D9E-416C-8130-CF745DFB0004},1100544,20211202003330,WANC,61.07872,-149.76603,utD2
3,LAS 7975,Dortch Bob A,Water Rights (801),Subsurface (SUB),Certificate Issued (36),LAS,7975,801,SUB,36,DORTCH,BOB,A,SUBSURFACE,CERT ISSUED,LAS 7975\nCERT ISSUED,Y,http://dnr.alaska.gov/projects/las/#filenumber...,"""2021-12-03T00:00:00.000Z""",{52C43B5B-2F86-4756-80DF-96B1D22C42D2},1100557,20211202003445,WANC,61.07386,-149.76479,utD3
4,LAS 9212,Mccoy John L,Water Rights (801),Subsurface (SUB),Certificate Issued (36),LAS,9212,801,SUB,36,MCCOY,JOHN,L,SUBSURFACE,CERT ISSUED,LAS 9212\nCERT ISSUED,Y,http://dnr.alaska.gov/projects/las/#filenumber...,"""2021-12-03T00:00:00.000Z""",{B6646D23-1E00-4331-9284-5FB5695FAF17},1101142,20211202003501,WANC,61.07417,-149.76467,utD4


In [4]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = pod['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "AKwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "AKwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "AKwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = ""
df['in_WaterSourceNativeID'] = "" # auto fill in below if not provdied
df['in_WaterSourceTypeCV'] = pod["CTSCLABEL"]

# Site Info
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = "Digitized"
df['in_County'] = ""
df['in_EPSGCodeCV'] = "4326"
df['in_Geometry'] = "" 
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = pod["latitude"]
df['in_Longitude'] = pod["longitude"]
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POD"  # "Point of Diversion"
df['in_SiteName'] = ""
df['in_SiteNativeID'] = "POD" + pod["CASE_ID"].astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = "WaDE Blank"
df['in_StateCV'] = "AK"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = ""
df['in_AllocationLegalStatusCV'] = pod["CS_LABEL"]
df['in_AllocationNativeID'] =  pod["MV_OID"]
df['in_AllocationOwner'] = pod["CSTMRNM"]
df['in_AllocationPriorityDate'] = ""
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = ""
df['in_AllocationTimeframeStart'] = ""
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = ""
df['in_BeneficialUseCategory'] = ""
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = "07/26/2023"
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = "1" # either a 1 or 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = "https://dnr.alaska.gov/mlw/water/rights/"

outPOD = df.copy()
outPOD = outPOD.drop_duplicates().reset_index(drop=True)
print(len(outPOD))
outPOD.head()

18879


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,utD0,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,59.68510,-151.47025,,,POD,,PODLAS 32578,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1111042,Bitter Daisy Lee,,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
1,utD1,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.07268,-149.77132,,,POD,,PODLAS 633,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1100529,Black Dennis J,,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
2,utD2,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.07872,-149.76603,,,POD,,PODLAS 7457,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1100544,Mierzejewski David W,,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
3,utD3,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.07386,-149.76479,,,POD,,PODLAS 7975,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1100557,Dortch Bob A,,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
4,utD4,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.07417,-149.76467,,,POD,,PODLAS 9212,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1101142,Mccoy John L,,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/


## Place of Use Data

In [5]:
# Input File - place of use data, Polygon.
PouSubSurfacePolygon = "shapefiles/SubsurfaceWaterRight_Polygon.csv"
PouPolygonFile = pd.read_csv(PouSubSurfacePolygon, encoding = "ISO-8859-1").replace(np.nan, "") 

PouSurfacePolygon = "shapefiles/SurfaceWaterRight_Polygon.csv"
PouSubPolygonFile = pd.read_csv(PouSurfacePolygon, encoding = "ISO-8859-1").replace(np.nan, "") 

PouSubPolygonFile= PouSubPolygonFile.groupby('CASE_ID').agg(lambda x: ','.join([str(elem) for elem in (list(set(x))) if elem != ""])).replace(np.nan, "").reset_index()   
pou = pd.concat([PouPolygonFile,PouSubPolygonFile], axis = 0)
pou = pou.reset_index(drop=True)

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in pou:
    pou['WaDEUUID'] = "utU" + pou.index.astype(str)
    pou.to_csv('CombinedPOU_Polygon.zip', compression=dict(method='zip', archive_name='CombinedPOU_Polygon.csv'), index=False)

print(len(pou))
pou.head()

46


,CASE_ID,CSTMRNM,CSTYPDSCRP,SPCLCDDSCR,CSSTTSDSCR,FILE_TYPE,FILENUMBER,CASE_TYPE,SPCLCD,CASESTATUS,CSTMRLSTNM,CSTMRFRSTN,CSTMRMDDLN,CTSCLABEL,CS_LABEL,MI_LABEL,DISPLAY,INFO_LINK,RFRSHDT,GLOBALID,LASTEDIT,MV_OID,OPR,Shape_Leng,Shape_Area,center_lat,center_lon,WaDEUUID
0,ADL 63463,Dnr Dopor Statewide,Water Rights (801),Subsurface (SUB),Cert. Pend. Action (38),ADL,63463,801,SUB,38,DNR DOPOR STATEWIDE,,,SUBSURFACE,CER PND ACTN,ADL 63463\nCER PND ACTN,Y,http://dnr.alaska.gov/projects/las/#filenumber...,"""2021-11-27T00:00:00.000Z""",{E8DDA4EC-0539-4626-A255-D16E6B6AF327},20140208174914,32,WFBX,0.00484,0.00000,61.57634,-150.60536,utU0
1,LAS 30407,"Pacrim Coal, Lp",Water Rights (801),Subsurface (SUB),Application Complete (11),LAS,30407,801,SUB,11,"PACRIM COAL, LP",,,SUBSURFACE,APLN,LAS 30407\nAPLN,Y,http://dnr.alaska.gov/projects/las/#filenumber...,"""2021-11-27T00:00:00.000Z""",{CC4AC1F7-956E-41B2-9D42-A724ED4AC65F},20151110193308,32244,WANC,0.25073,0.00196,61.19364,-151.40482,utU1
2,LAS 30407,"Pacrim Coal, Lp",Water Rights (801),Subsurface (SUB),Application Complete (11),LAS,30407,801,SUB,11,"PACRIM COAL, LP",,,SUBSURFACE,APLN,LAS 30407\nAPLN,Y,http://dnr.alaska.gov/projects/las/#filenumber...,"""2021-11-27T00:00:00.000Z""",{A8BCD5B1-298E-4B20-B651-BF521D3B3637},20151110183737,32241,WANC,0.08579,0.00034,61.18584,-151.35580,utU2
3,ADL 59683,Monroe W Bruce,Water Rights (801),Surface (SUR),Certificate Issued (36),ADL,59683,801,SUR,36,MONROE,W,BRUCE,SURFACE,CERT ISSUED,ADL 59683\nCERT ISSUED,Y,http://dnr.alaska.gov/projects/las/#filenumber...,"""2021-11-27T00:00:00.000Z""",{2409F03E-F14D-4295-AE7F-CC9FE7F016D8},20140208175030,32,WJNO,0.00879912944149,3.4999308792e-06,62.3073696197,-150.73148578,utU3
4,LAS 29737,"Pacrim Coal, Lp",Water Rights (801),Surface (SUR),Postponed/Suspended (60),LAS,29737,801,SUR,60,"PACRIM COAL, LP",,,SURFACE,SUSPEND,LAS 29737\nSUSPEND,Y,http://dnr.alaska.gov/projects/las/#filenumber...,"""2021-11-27T00:00:00.000Z""",{946BB0A6-8FB9-4AEC-9F69-C95BA5E49834},20140725215508,33201,WANC,0.0015894857091,1.72444631717e-07,61.2190916447,-151.389712637,utU4


In [6]:
# create output POU dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = pou['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "AKwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "AKwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "AKwr_O1"

# WaterSource Info
df['in_Geometry'] = "" 
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = ""
df['in_WaterSourceNativeID'] = ""
df['in_WaterSourceTypeCV'] = pou['CTSCLABEL']

# Site Info
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = "Digitized"
df['in_County'] = ""
df['in_EPSGCodeCV'] = "4326"
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = pou["center_lat"]
df['in_Longitude'] = pou["center_lon"]
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POU"
df['in_SiteName'] = ""
df['in_SiteNativeID'] = "POU" + pou["CASE_ID"].astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = "WaDE Blank"
df['in_StateCV'] = "AK"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = ""
df['in_AllocationLegalStatusCV'] = pou["CS_LABEL"]
df['in_AllocationNativeID'] =  pou["MV_OID"]
df['in_AllocationOwner'] = pou["CSTMRNM"]
df['in_AllocationPriorityDate'] = ""
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = ""
df['in_AllocationTimeframeStart'] = ""
df['in_AllocationTypeCV'] = pou["SPCLCDDSCR"]
df['in_AllocationVolume_AF'] = ""
df['in_BeneficialUseCategory'] = ""
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = "07/26/2023"
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = "1" # either a 1 or 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = "https://dnr.alaska.gov/mlw/water/rights/"

outPOU = df.copy()
outPOU = outPOU.drop_duplicates().reset_index(drop=True)
print(len(outPOU))
outPOU.head()

46


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,utU0,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.57634,-150.60536,,,POU,,POUADL 63463,,WaDE Blank,AK,,,,,,,,,,,CER PND ACTN,32,Dnr Dopor Statewide,,,,,Subsurface (SUB),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
1,utU1,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.19364,-151.40482,,,POU,,POULAS 30407,,WaDE Blank,AK,,,,,,,,,,,APLN,32244,"Pacrim Coal, Lp",,,,,Subsurface (SUB),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
2,utU2,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.18584,-151.35580,,,POU,,POULAS 30407,,WaDE Blank,AK,,,,,,,,,,,APLN,32241,"Pacrim Coal, Lp",,,,,Subsurface (SUB),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
3,utU3,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SURFACE,,Digitized,,4326,,,,62.3073696197,-150.73148578,,,POU,,POUADL 59683,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,32,Monroe W Bruce,,,,,Surface (SUR),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
4,utU4,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SURFACE,,Digitized,,4326,,,,61.2190916447,-151.389712637,,,POU,,POULAS 29737,,WaDE Blank,AK,,,,,,,,,,,SUSPEND,33201,"Pacrim Coal, Lp",,,,,Surface (SUR),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/


## Concatenate POD and POU Data.  Make needed changes

In [7]:
# Concatenate dataframes
frames = [outPOD, outPOU]
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

18925


In [8]:
outdf['in_AllocationLegalStatusCV'].unique()

array(['CERT ISSUED', 'PER PND ACTN', 'ISSUED', 'CER PND ACTN', 'APLN',
       'SUSPEND', 'ADD INFO REQ', 'ACTV ADJDCTN'], dtype=object)

In [9]:
outdf

,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,utD0,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,59.68510,-151.47025,,,POD,,PODLAS 32578,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1111042,Bitter Daisy Lee,,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
1,utD1,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.07268,-149.77132,,,POD,,PODLAS 633,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1100529,Black Dennis J,,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
2,utD2,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.07872,-149.76603,,,POD,,PODLAS 7457,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1100544,Mierzejewski David W,,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
3,utD3,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.07386,-149.76479,,,POD,,PODLAS 7975,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1100557,Dortch Bob A,,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
4,utD4,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SUBSURFACE,,Digitized,,4326,,,,61.07417,-149.76467,,,POD,,PODLAS 9212,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1101142,Mccoy John L,,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,utU41,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,SURFACE,,Digitized,,4326,,,,62.039951007,-158.216503202,,,POU,,POULAS 31477,,WaDE Blank,AK,,,,,,,,,,,ISSUED,39601,Donlin Gold Llc,,,,,Surface (SUR),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
18921,utU42,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,,,Digitized,,4326,,,,68.0576015505,-162.866122149,,,POU,,POULAS 33141,,WaDE Blank,AK,,,,,,,,,,,ISSUED,310241,Teck Alaska Incorporated,,,,,Surface (SUR),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
18922,utU43,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,,,Digitized,,4326,,,,68.0692149962,-162.831258975,,,POU,,POULAS 33147,,WaDE Blank,AK,,,,,,,,,,,ISSUED,310242,Teck Alaska Incorporated,,,,,Surface (SUR),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
18923,utU44,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,,,Digitized,,4326,,,,68.044532976,-162.847324314,,,POU,,POULAS 33151,,WaDE Blank,AK,,,,,,,,,,,ISSUED,310243,Teck Alaska Incorporated,,,,,Surface (SUR),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/


## Clean Data / data types

In [10]:
# Clean name entries of spcial characters
def removeSpecialCharsFunc(Val):
    Val = str(Val)
    Val = re.sub("[$@&.;/\)(-]", "", Val).title().replace("  ", " ").strip()
    return Val

In [11]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

array([''], dtype=object)

In [12]:
outdf['in_SiteName'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array([''], dtype=object)

In [13]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

array(['Bitter Daisy Lee', 'Black Dennis J', 'Mierzejewski David W', ...,
       'Flora Michael William', 'Williams John V A', 'Pacrim Coal, Lp'],
      dtype=object)

In [14]:
# Ensure Empty String / remove string value of "nan"

def ensureEmptyString(val):
    val = str(val).strip()
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = ""
    else:
        outString = val
    return outString

In [15]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

array([''], dtype=object)

In [16]:
outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

array(['SUBSURFACE', '', 'SURFACE'], dtype=object)

In [17]:
outdf['in_SiteTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

array(['WaDE Blank'], dtype=object)

In [18]:
outdf['in_SiteName'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array([''], dtype=object)

In [19]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

array(['Bitter Daisy Lee', 'Black Dennis J', 'Mierzejewski David W', ...,
       'Flora Michael William', 'Williams John V A', 'Pacrim Coal, Lp'],
      dtype=object)

In [20]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: ensureEmptyString(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

array([''], dtype=object)

In [21]:
# Ensure Latitude entry is either numireic or blank, no 0 entries
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').replace(0,"").fillna("")
outdf['in_Latitude'].unique()

array([59.685095931, 61.072679952, 61.078715314, ..., 68.0692149962,
       68.044532976, 68.0649729389], dtype=object)

In [22]:
# Ensure Longitude entry is either numireic or blank, no 0 entries
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').replace(0,"").fillna("")
outdf['in_Longitude'].unique()

array([-151.470247006, -149.771321703, -149.766026603, ...,
       -162.831258975, -162.847324314, -162.798780983], dtype=object)

In [23]:
# Changing datatype of Priority Date to date fields entry
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'], errors = 'coerce')
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf["in_AllocationPriorityDate"].dt.strftime('%m/%d/%Y'))
outdf['in_AllocationPriorityDate'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [24]:
# Ensure Flow entry is either numireic or blank, no 0 entries
outdf['in_AllocationFlow_CFS'] = pd.to_numeric(outdf['in_AllocationFlow_CFS'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationFlow_CFS'].unique()

array([''], dtype=object)

In [25]:
# Ensure Volume entry is either numireic or blank, no 0 entries
outdf['in_AllocationVolume_AF'] = pd.to_numeric(outdf['in_AllocationVolume_AF'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationVolume_AF'].unique()

array([''], dtype=object)

In [26]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)
dfWaterSourceNativeID['linkKey'] = dfWaterSourceNativeID['in_WaterSourceName'].astype(str) + dfWaterSourceNativeID['in_WaterSourceTypeCV'].astype(str)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
WaterSourceNativeIDdict = pd.Series(dfWaterSourceNativeID.in_WaterSourceNativeID.values, index=dfWaterSourceNativeID.linkKey.astype(str)).to_dict()
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        colrowValue = str(A).strip() + str(B).strip()
        try:
            outList = WaterSourceNativeIDdict[colrowValue]
        except:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

array(['wadeID1', '', 'wadeID3'], dtype=object)

## Drop non-Active AllocationLegalStatusCV Water Rights
- For this {state name / organization}, we don't want water rights that are considered: {enter string entries here}

In [27]:
# drop non-active AllocationLegalStatusCV values specific to that state.

# drop the list
dropLegalStatusList = ["SUSPEND", "ADD INFO REQ"] # enter string entries here

# drop rows from above list
outdf = outdf[outdf.in_AllocationLegalStatusCV.isin(dropLegalStatusList) == False].reset_index(drop=True)

print(len(outdf))
outdf['in_AllocationLegalStatusCV'].unique()

18768


array(['CERT ISSUED', 'PER PND ACTN', 'ISSUED', 'CER PND ACTN', 'APLN',
       'ACTV ADJDCTN'], dtype=object)

## Shapefile Data
- For attaching geometry to POU csv inputs.

In [29]:
# PoU Shapefile Data    Combining both shape files into one
shapefileInputSub = "shapefiles/SubsurfaceWaterRight_Polygon.zip" # ziped folder of the shp file
dfPoUshapetempSub = gpd.read_file(shapefileInputSub)

shapefileInputSur = "shapefiles/SurfaceWaterRight_Polygon.zip" # ziped folder of the shp file
dfPoUshapetempSur = gpd.read_file(shapefileInputSur)

dfPoUshapetemp = pd.concat([dfPoUshapetempSub,dfPoUshapetempSur], axis = 0)
dfPoUshapetemp = dfPoUshapetemp.reset_index(drop=True)

print(len(dfPoUshapetemp))
dfPoUshapetemp.head()

59


,CASE_ID,CSTMRNM,CSTYPDSCRP,SPCLCDDSCR,CSSTTSDSCR,FILE_TYPE,FILENUMBER,CASE_TYPE,SPCLCD,CASESTATUS,CSTMRLSTNM,CSTMRFRSTN,CSTMRMDDLN,CTSCLABEL,CS_LABEL,MI_LABEL,DISPLAY,INFO_LINK,RFRSHDT,GLOBALID,LASTEDIT,MV_OID,OPR,Shape_Leng,Shape_Area,center_lat,center_lon,geometry
0,ADL 63463,Dnr Dopor Statewide,Water Rights (801),Subsurface (SUB),Cert. Pend. Action (38),ADL,63463,801,SUB,38,DNR DOPOR STATEWIDE,None,None,SUBSURFACE,CER PND ACTN,ADL 63463\nCER PND ACTN,Y,http://dnr.alaska.gov/projects/las/#filenumber...,2021-11-27,{E8DDA4EC-0539-4626-A255-D16E6B6AF327},20140208174914,32,WFBX,0.00484,0.00000,61.57634,-150.60536,"POLYGON ((-150.60574 61.57730, -150.60548 61.5..."
1,LAS 30407,"Pacrim Coal, Lp",Water Rights (801),Subsurface (SUB),Application Complete (11),LAS,30407,801,SUB,11,"PACRIM COAL, LP",None,None,SUBSURFACE,APLN,LAS 30407\nAPLN,Y,http://dnr.alaska.gov/projects/las/#filenumber...,2021-11-27,{CC4AC1F7-956E-41B2-9D42-A724ED4AC65F},20151110193308,32244,WANC,0.25073,0.00196,61.19364,-151.40482,"POLYGON ((-151.41814 61.16882, -151.42266 61.1..."
2,LAS 30407,"Pacrim Coal, Lp",Water Rights (801),Subsurface (SUB),Application Complete (11),LAS,30407,801,SUB,11,"PACRIM COAL, LP",None,None,SUBSURFACE,APLN,LAS 30407\nAPLN,Y,http://dnr.alaska.gov/projects/las/#filenumber...,2021-11-27,{A8BCD5B1-298E-4B20-B651-BF521D3B3637},20151110183737,32241,WANC,0.08579,0.00034,61.18584,-151.35580,"POLYGON ((-151.35663 61.19125, -151.35591 61.1..."
3,LAS 29751,"Pacrim Coal, Lp",Water Rights (801),Surface (SUR),Postponed/Suspended (60),LAS,29751,801,SUR,60,"PACRIM COAL, LP",None,None,SURFACE,SUSPEND,LAS 29751\nSUSPEND,Y,http://dnr.alaska.gov/projects/las/#filenumber...,2021-11-27,{B575A4D4-46F3-4DFE-8054-76E15F66D7C3},20140728163702,33530,WANC,0.00253,0.00000,61.20539,-151.39609,"POLYGON ((-151.39629 61.20512, -151.39659 61.2..."
4,LAS 29758,"Pacrim Coal, Lp",Water Rights (801),Surface (SUR),Postponed/Suspended (60),LAS,29758,801,SUR,60,"PACRIM COAL, LP",None,None,SURFACE,SUSPEND,LAS 29758\nSUSPEND,None,http://dnr.alaska.gov/projects/las/#filenumber...,2021-11-27,{E41087C2-8F66-4FA7-A0D1-7D3572F71B04},20140717192138,31608,WANC,0.00422,0.00000,61.18949,-151.41104,"POLYGON ((-151.41085 61.18969, -151.41026 61.1..."


In [30]:
# create temp dataframe to hold native ID and geometry from shapefile input
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)

# assing values to temp dataframe based on shapefile input
# for in_SiteNativeID assure ID value is the same as that listed above for POU info.
dfPoUshape['in_SiteNativeID'] = "POU" + pou["CASE_ID"].astype(str)
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
print(len(dfPoUshape))
dfPoUshape.head()

46


,in_SiteNativeID,geometry
0,POUADL 63463,"POLYGON ((-150.60574 61.57730, -150.60548 61.5..."
1,POULAS 30407,"POLYGON ((-151.41814 61.16882, -151.42266 61.1..."
2,POULAS 30407,"POLYGON ((-151.35663 61.19125, -151.35591 61.1..."
3,POUADL 59683,"POLYGON ((-151.39629 61.20512, -151.39659 61.2..."
4,POULAS 29737,"POLYGON ((-151.41085 61.18969, -151.41026 61.1..."


## Export Data

In [31]:
outdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18768 entries, 0 to 18767
Data columns (total 63 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   WaDEUUID                                      18768 non-null  object        
 1   in_MethodUUID                                 18768 non-null  object        
 2   in_VariableSpecificUUID                       18768 non-null  object        
 3   in_OrganizationUUID                           18768 non-null  object        
 4   in_Geometry                                   18768 non-null  object        
 5   in_GNISFeatureNameCV                          18768 non-null  object        
 6   in_WaterQualityIndicatorCV                    18768 non-null  object        
 7   in_WaterSourceName                            18768 non-null  object        
 8   in_WaterSourceNativeID                        18768 non-null  obje

In [32]:
outdf

,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,utD0,AKwr_M1,AKwr_V1,AKwr_O1,,,,,wadeID1,SUBSURFACE,,Digitized,,4326,,,,59.68510,-151.47025,,,POD,,PODLAS 32578,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1111042,Bitter Daisy Lee,NaT,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
1,utD1,AKwr_M1,AKwr_V1,AKwr_O1,,,,,wadeID1,SUBSURFACE,,Digitized,,4326,,,,61.07268,-149.77132,,,POD,,PODLAS 633,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1100529,Black Dennis J,NaT,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
2,utD2,AKwr_M1,AKwr_V1,AKwr_O1,,,,,wadeID1,SUBSURFACE,,Digitized,,4326,,,,61.07872,-149.76603,,,POD,,PODLAS 7457,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1100544,Mierzejewski David W,NaT,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
3,utD3,AKwr_M1,AKwr_V1,AKwr_O1,,,,,wadeID1,SUBSURFACE,,Digitized,,4326,,,,61.07386,-149.76479,,,POD,,PODLAS 7975,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1100557,Dortch Bob A,NaT,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
4,utD4,AKwr_M1,AKwr_V1,AKwr_O1,,,,,wadeID1,SUBSURFACE,,Digitized,,4326,,,,61.07417,-149.76467,,,POD,,PODLAS 9212,,WaDE Blank,AK,,,,,,,,,,,CERT ISSUED,1101142,Mccoy John L,NaT,,,,,,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18763,utU41,AKwr_M1,AKwr_V1,AKwr_O1,,,,,wadeID3,SURFACE,,Digitized,,4326,,,,62.03995,-158.21650,,,POU,,POULAS 31477,,WaDE Blank,AK,,,,,,,,,,,ISSUED,39601,Donlin Gold Llc,NaT,,,,Surface (SUR),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
18764,utU42,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,,,Digitized,,4326,,,,68.05760,-162.86612,,,POU,,POULAS 33141,,WaDE Blank,AK,,,,,,,,,,,ISSUED,310241,Teck Alaska Incorporated,NaT,,,,Surface (SUR),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
18765,utU43,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,,,Digitized,,4326,,,,68.06921,-162.83126,,,POU,,POULAS 33147,,WaDE Blank,AK,,,,,,,,,,,ISSUED,310242,Teck Alaska Incorporated,NaT,,,,Surface (SUR),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/
18766,utU44,AKwr_M1,AKwr_V1,AKwr_O1,,,,,,,,Digitized,,4326,,,,68.04453,-162.84732,,,POU,,POULAS 33151,,WaDE Blank,AK,,,,,,,,,,,ISSUED,310243,Teck Alaska Incorporated,NaT,,,,Surface (SUR),,,,,,07/26/2023,,1,,,,,,,,,,https://dnr.alaska.gov/mlw/water/rights/


In [34]:
# Export the output dataframe
# change output name / abbreviation to match native state provdier and wade data type 
outdf.to_csv('Pwr_akMain.zip', compression=dict(method='zip', archive_name='Pwr_utMain.csv'), index=False)  # The output, save as a zip
dfPoUshape.to_csv('P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.